# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-d68b77ae1d-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Euro explorative search") 

Consider the following exploratory information need:

> investigate the Euro currency, which are the countries that currently use the Euro and which were their currencies before the Euro. Take a look also to other currencies and find which are the most used.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q38`| Italy   | node |





Also consider

```
wd:Q38 ?p ?obj .
```

is the BGP to retrieve all **properties of Italy**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for currency

2. Return the countries which currently use Euro

3. Return the countries belonging to the European Union, with a currency different from the Euro (return also their currency).

4. For each country that now uses Euro return which was the currencies used before

5. Return all types of dollar in the world (US Dollar, Australian Dollar, ecc...)

6. Return the currencies used in at least 2 different countries (return also the number of countries using each currency).

7. Consider the currencies obtained in task 6, for each currency return the number of people using that currency (hint: consider the maximum popolation available for each country).

## Task 1
Identify the BGP for currency

In [3]:
# query example
queryString = """
SELECT DISTINCT ?p ?name ?value
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   ?p sc:name ?name .
   ?obj sc:name ?value .
}
# add limit 200 to show currency
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('name', 'member of'), ('value', 'European Bank for Reconstruction and Development')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('name', 'language used'), ('value', 'Sicilian')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('name', 'history of topic'), ('value', 'history of Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P163'), ('name', 'flag'), ('value', 'flag of Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('name', 'contains administrative territorial entity'), ('value', 'Tuscany')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('name', 'language used'), ('value', 'Neapolitan')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('name', 'member of'), ('value', 'Inter-American Development Bank')]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('name', 'contains administrative territorial entity'), ('value', 'Trentino-South Tyrol')]
[('p', 'http://www.wikidata.org/prop/dir

wdt:P38 is the currency IRI

In [4]:
queryString = """ 
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?p wdt:P38 ?obj .  
   # get the label
   ?p sc:name ?name .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q491559'), ('name', 'First Republic of South Korea')]
[('p', 'http://www.wikidata.org/entity/Q5148320'), ('name', 'Colonial Fiji')]
[('p', 'http://www.wikidata.org/entity/Q1649306'), ('name', 'Nyasaland')]
[('p', 'http://www.wikidata.org/entity/Q488521'), ('name', 'Fourth Republic of South Korea')]
[('p', 'http://www.wikidata.org/entity/Q699446'), ('name', 'Dutch Formosa')]
[('p', 'http://www.wikidata.org/entity/Q17623840'), ('name', 'CdZ-Gebiet Elsaß')]
[('p', 'http://www.wikidata.org/entity/Q783684'), ('name', 'Provisional Government of the French Republic')]
[('p', 'http://www.wikidata.org/entity/Q20982493'), ('name', 'Central Regions State')]
[('p', 'http://www.wikidata.org/entity/Q21816225'), ('name', 'Protectorate of Bahrain')]
[('p', 'http://www.wikidata.org/entity/Q868942'), ('name', 'Republic of Kuwait')]
[('p', 'http://www.wikidata.org/entity/Q2499687'), ('name', 'Yatton')]
[('p', 'http://www.wikidata.org/entity/Q158236'), ('name

In [5]:
queryString = """ 
SELECT ?p ?name 
WHERE {
   # bind something
   wd:Q38 wdt:P31 ?p . #Italy is intance of
   # get the label
   ?p sc:name ?name .
}
# GROUP BY ?inst ?iname
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('p', 'http://www.wikidata.org/entity/Q6256'), ('name', 'country')]
2


In [6]:
queryString = """ 
SELECT ?p ?name
WHERE {
   # bind something
   ?p wdt:P1647  ?obj .
   ?obj ?p wdt:P38 .
   # get the label
   ?p sc:name ?name .
}
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
Empty


In [7]:
queryString = """ 
SELECT ?p ?name
WHERE {
   # bind something
   wdt:P38 wdt:P27 ?p .
   # get the label
   ?p sc:name ?name .
}
LIMIT 20
""" 
print("Results")
x=run_query(queryString)     

Results
Empty


In [8]:
queryString = """ 
SELECT ?p ?name
WHERE {
   wd:Q38 wdt:P38 ?p .
   # get the label
   ?p sc:name ?name .
}
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q4916'), ('name', 'euro')]
1


In [9]:
queryString = """ 
SELECT ?p ?name
WHERE {
   wd:Q4916 wdt:P31 ?p .
   # get the label
   ?p sc:name ?name .
}
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q28795800'), ('name', 'decimal currency')]
[('p', 'http://www.wikidata.org/entity/Q8142'), ('name', 'currency')]
2


In [4]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q8142 ?p ?obj .
   # get the label
   ?p sc:name ?name .
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1482'), ('name', 'Stack Exchange tag')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2184')

In [11]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q6256 wdt:P38 ?p .
   # get the label
   ?p sc:name ?name .
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [12]:
queryString = """
SELECT DISTINCT ?p ?name 
WHERE 
{
  ?p wdt:P31 wd:Q6256 .      # all countries   
  ?p wdt:P38 wd:Q4916 .   # have currency euro   
  ?p sc:name ?name .
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('p', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('p', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('p', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('p', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('p', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('p', 'http://www.wikidata.org/entity/Q236'), ('name', 'Montenegro')]
[('p', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('p', 'http://www.wikidata.org/entity/Q29999'), ('name', 'Kingdom of the Netherlands')]
[('p', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland')]
[('p', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('p', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria')]
[('p', 'http://www.wikidata.org/entity/Q41'), ('name', 'Greece')]
[('p', 'http://www.wikidata.org/entity/Q45'), ('name', 'Portugal')]
[('p', 'http://www.wikidata.

In [13]:
queryString = """ 
SELECT ?p ?name
WHERE {
   wd:Q4916 ?p ?obj .
   # get the label
   ?p sc:name ?name .
}
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('name', 'applies to jurisdiction')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/

In [4]:
queryString = """ 
SELECT ?p ?name
WHERE {
   wd:Q8142 wdt:P31 ?p .
   # get the label
   ?p sc:name ?name .
}
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
Empty


In [5]:
queryString = """ 
SELECT ?p ?name
WHERE {
   wd:Q4916 wdt:P31 ?p .
   # get the label
   ?p sc:name ?name .
}
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q28795800'), ('name', 'decimal currency')]
[('p', 'http://www.wikidata.org/entity/Q8142'), ('name', 'currency')]
2


Final query for this task

In [2]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q8142 ?p ?obj .
   # get the label
   ?p sc:name ?name .
}
# add limit 200 to show currency
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1482'), ('name', 'Stack Exchange tag')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2184')

## Task 2
Return the countries which currently use Euro

In [9]:
queryString = """
SELECT DISTINCT ?s ?name 
WHERE 
{
  ?s wdt:P31 wd:Q6256 .      # all countries   
  ?s wdt:P38 wd:Q4916 .      # have currency euro 
  ?s sc:name ?name .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('s', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('s', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('s', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('s', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('s', 'http://www.wikidata.org/entity/Q236'), ('name', 'Montenegro')]
[('s', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('s', 'http://www.wikidata.org/entity/Q29999'), ('name', 'Kingdom of the Netherlands')]
[('s', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland')]
[('s', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('s', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria')]
[('s', 'http://www.wikidata.org/entity/Q41'), ('name', 'Greece')]
[('s', 'http://www.wikidata.org/entity/Q45'), ('name', 'Portugal')]
[('s', 'http://www.wikidata.

Final query for this task

In [10]:
# write your final query
queryString = """
SELECT DISTINCT ?s ?name 
WHERE 
{
  ?s wdt:P31 wd:Q6256 .      # all countries   
  ?s wdt:P38 wd:Q4916 .      # have currency euro 
  ?s sc:name ?name .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('s', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('s', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('s', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('s', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('s', 'http://www.wikidata.org/entity/Q236'), ('name', 'Montenegro')]
[('s', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('s', 'http://www.wikidata.org/entity/Q29999'), ('name', 'Kingdom of the Netherlands')]
[('s', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland')]
[('s', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('s', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria')]
[('s', 'http://www.wikidata.org/entity/Q41'), ('name', 'Greece')]
[('s', 'http://www.wikidata.org/entity/Q45'), ('name', 'Portugal')]
[('s', 'http://www.wikidata.

## Task 3
Return the countries belonging to the European Union, with a currency different from the Euro (return also their currency).

In [18]:
# write your queries
# Italy is member of
queryString = """ 
SELECT ?p ?name
WHERE {
   wd:Q38 wdt:P463 ?p .
   # get the label
   ?p sc:name ?name .
}
LIMIT 40
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q488981'), ('name', 'European Bank for Reconstruction and Development')]
[('p', 'http://www.wikidata.org/entity/Q1153087'), ('name', 'Inter-American Development Bank')]
[('p', 'http://www.wikidata.org/entity/Q848633'), ('name', 'International Chamber of Commerce')]
[('p', 'http://www.wikidata.org/entity/Q1043527'), ('name', 'Multilateral Investment Guarantee Agency')]
[('p', 'http://www.wikidata.org/entity/Q1065'), ('name', 'United Nations')]
[('p', 'http://www.wikidata.org/entity/Q1072120'), ('name', 'Missile Technology Control Regime')]
[('p', 'http://www.wikidata.org/entity/Q123209'), ('name', 'Latin Union')]
[('p', 'http://www.wikidata.org/entity/Q125761'), ('name', 'International Civil Aviation Organization')]
[('p', 'http://www.wikidata.org/entity/Q1377612'), ('name', 'European Payments Union')]
[('p', 'http://www.wikidata.org/entity/Q141720'), ('name', 'European Air Transport Command')]
[('p', 'http://www.wikidata.org/entity/Q148079

In [19]:
# write your queries
#Countries in Europian Union that dont have euro as currency
queryString = """ 
SELECT ?p ?name ?currency ?currencyLabel
WHERE {
  ?p wdt:P31 wd:Q6256 . # Must be of a country
  ?p wdt:P463 wd:Q458 . # Must be member of European Union 
  FILTER not exists {
    ?p wdt:P38 wd:Q4916 . # exclude countries that have euro as currency 
  }
  ?p wdt:P38 ?currency .
  ?p sc:name ?name .
  ?currency sc:name ?currencyLabel .
}
LIMIT 40
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q36'), ('name', 'Poland'), ('currency', 'http://www.wikidata.org/entity/Q123213'), ('currencyLabel', 'złoty')]
[('p', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria'), ('currency', 'http://www.wikidata.org/entity/Q172540'), ('currencyLabel', 'Bulgarian lev')]
[('p', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary'), ('currency', 'http://www.wikidata.org/entity/Q47190'), ('currencyLabel', 'forint')]
[('p', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania'), ('currency', 'http://www.wikidata.org/entity/Q131645'), ('currencyLabel', 'Romanian leu')]
[('p', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('currency', 'http://www.wikidata.org/entity/Q25224'), ('currencyLabel', 'pound sterling')]
[('p', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia'), ('currency', 'http://www.wikidata.org/entity/Q26360'), ('currencyLabel', 'Croatian kuna')]
[('p', 'http://www.wikidata.org/entity/Q213')

Final query for this task

In [20]:
#Countries in Europian Union that dont have euro as currency
queryString = """ 
SELECT ?p ?name ?currencyLabel
WHERE {
  ?p wdt:P31 wd:Q6256 . # Must be of a country
  ?p wdt:P463 wd:Q458 . # Must be member of European Union 
  FILTER not exists {
    ?p wdt:P38 wd:Q4916 . # exclude countries that have euro as currency 
  }
  ?p wdt:P38 ?currency .
  ?p sc:name ?name .
  ?currency sc:name ?currencyLabel .
}
LIMIT 40
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q36'), ('name', 'Poland'), ('currencyLabel', 'złoty')]
[('p', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria'), ('currencyLabel', 'Bulgarian lev')]
[('p', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary'), ('currencyLabel', 'forint')]
[('p', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania'), ('currencyLabel', 'Romanian leu')]
[('p', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('currencyLabel', 'pound sterling')]
[('p', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia'), ('currencyLabel', 'Croatian kuna')]
[('p', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic'), ('currencyLabel', 'Czech koruna')]
[('p', 'http://www.wikidata.org/entity/Q35'), ('name', 'Denmark'), ('currencyLabel', 'Danish krone')]
[('p', 'http://www.wikidata.org/entity/Q34'), ('name', 'Sweden'), ('currencyLabel', 'Swedish krona')]
9


## Task 4
For each country that now uses Euro return which was the currencies used before

In [4]:
# write your queries
queryString = """ 
SELECT ?currency ?currencyLabel
WHERE {
  wd:Q142 wdt:P38 ?currency .
  ?currency sc:name ?currencyLabel .
}
LIMIT 40
"""
print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyLabel', 'euro')]
1


Final query for this task

In [ ]:
# write your final query

## Task 5
Return all types of dollar in the world (US Dollar, Australian Dollar, ecc...)

In [21]:
#returning all currencies 
queryString = """ 
SELECT ?country ?countryName ?currency ?currencyLabel
WHERE {
    ?country wdt:P31 wd:Q6256 .
    ?country wdt:P38 ?currency .
    # get the label
    ?country sc:name ?countryName .
    ?currency sc:name ?currencyLabel .
}
LIMIT 40
"""
print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q1011'), ('countryName', 'Cape Verde'), ('currency', 'http://www.wikidata.org/entity/Q4591'), ('currencyLabel', 'Cape Verdean escudo')]
[('country', 'http://www.wikidata.org/entity/Q36'), ('countryName', 'Poland'), ('currency', 'http://www.wikidata.org/entity/Q123213'), ('currencyLabel', 'złoty')]
[('country', 'http://www.wikidata.org/entity/Q79'), ('countryName', 'Egypt'), ('currency', 'http://www.wikidata.org/entity/Q199462'), ('currencyLabel', 'Egyptian pound')]
[('country', 'http://www.wikidata.org/entity/Q833'), ('countryName', 'Malaysia'), ('currency', 'http://www.wikidata.org/entity/Q163712'), ('currencyLabel', 'Malaysian ringgit')]
[('country', 'http://www.wikidata.org/entity/Q846'), ('countryName', 'Qatar'), ('currency', 'http://www.wikidata.org/entity/Q206386'), ('currencyLabel', 'Qatari riyal')]
[('country', 'http://www.wikidata.org/entity/Q766'), ('countryName', 'Jamaica'), ('currency', 'http://www.wikidata.org/entity/Q20

In [22]:
#returning all currencies that contain dollar in their name 
queryString = """ 
SELECT ?country ?countryName ?currency ?currencyLabel
WHERE {
    ?country wdt:P31 wd:Q6256 .
    ?country wdt:P38 ?currency .
    # get the label
    ?country sc:name ?countryName .
    ?currency sc:name ?currencyLabel .
    FILTER (REGEX(STR(?currencyLabel), ".*dollar")) .
}

LIMIT 40
"""
print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q16644'), ('countryName', 'Northern Mariana Islands'), ('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyLabel', 'United States dollar')]
[('country', 'http://www.wikidata.org/entity/Q1014'), ('countryName', 'Liberia'), ('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyLabel', 'United States dollar')]
[('country', 'http://www.wikidata.org/entity/Q1014'), ('countryName', 'Liberia'), ('currency', 'http://www.wikidata.org/entity/Q242988'), ('currencyLabel', 'Liberian dollar')]
[('country', 'http://www.wikidata.org/entity/Q1030'), ('countryName', 'Namibia'), ('currency', 'http://www.wikidata.org/entity/Q202462'), ('currencyLabel', 'Namibian dollar')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('countryName', 'Canada'), ('currency', 'http://www.wikidata.org/entity/Q1104069'), ('currencyLabel', 'Canadian dollar')]
[('country', 'http://www.wikidata.org/entity/Q16641'), ('countryName', 'American Samoa'), 

Final query for this task

In [23]:
queryString = """ 
SELECT DISTINCT ?currency ?currencyLabel
WHERE {
    ?country wdt:P31 wd:Q6256 .
    ?country wdt:P38 ?currency .
    # get the label
    ?country sc:name ?countryName .
    ?currency sc:name ?currencyLabel .
    FILTER (REGEX(STR(?currencyLabel), ".*[Dd]ollar*.")) .
}
LIMIT 40
"""
print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyLabel', 'United States dollar')]
[('currency', 'http://www.wikidata.org/entity/Q242988'), ('currencyLabel', 'Liberian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q202462'), ('currencyLabel', 'Namibian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q1104069'), ('currencyLabel', 'Canadian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q194351'), ('currencyLabel', 'Barbadian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q1472704'), ('currencyLabel', 'New Zealand dollar')]
[('currency', 'http://www.wikidata.org/entity/Q507737'), ('currencyLabel', 'Cook Islands dollar')]
[('currency', 'http://www.wikidata.org/entity/Q190951'), ('currencyLabel', 'Singapore dollar')]
[('currency', 'http://www.wikidata.org/entity/Q4165057'), ('currencyLabel', 'Niue dollar')]
[('currency', 'http://www.wikidata.org/entity/Q259502'), ('currencyLabel', 'Australian dollar')]
[('currency', 'http://www.wik

## Task 6
Return the currencies used in at least 2 different countries (return also the number of countries using each currency).

In [24]:
#counting how many countries use each currency 
queryString = """ 
SELECT ?currency ?currencyLabel (COUNT(?country) AS ?countries)
WHERE {
    # country wdt:P31 wd:Q6256 .
    ?country wdt:P38 ?currency .
    # get the label
    # ?country sc:name ?countryName .
    ?currency sc:name ?currencyLabel .
    
}

GROUP BY ?currency ?currencyLabel
LIMIT 40
"""
print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q16068'), ('currencyLabel', 'Deutsche Mark'), ('countries', '9')]
[('currency', 'http://www.wikidata.org/entity/Q3712659'), ('currencyLabel', 'Texas Dollar'), ('countries', '1')]
[('currency', 'http://www.wikidata.org/entity/Q201875'), ('currencyLabel', 'kyat'), ('countries', '4')]
[('currency', 'http://www.wikidata.org/entity/Q305981'), ('currencyLabel', 'Abbasi'), ('countries', '5')]
[('currency', 'http://www.wikidata.org/entity/Q173117'), ('currencyLabel', 'Brazilian real'), ('countries', '5')]
[('currency', 'http://www.wikidata.org/entity/Q200055'), ('currencyLabel', 'Lao kip'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q8192962'), ('currencyLabel', 'Eiraku Tsūhō'), ('countries', '1')]
[('currency', 'http://www.wikidata.org/entity/Q3931076'), ('currencyLabel', 'Central American Republic real'), ('countries', '1')]
[('currency', 'http://www.wikidata.org/entity/Q193094'), ('currencyLabel', 'Iraqi dinar'), (

Final query for this task

In [6]:
# write your final query
queryString = """ 
SELECT ?currency ?currencyLabel (COUNT(?country) AS ?countries)
WHERE {
    # country wdt:P31 wd:Q6256 .
    ?country wdt:P38 ?currency .
    # get the label
    # ?country sc:name ?countryName .
    ?currency sc:name ?currencyLabel .
    
}

GROUP BY ?currency ?currencyLabel
HAVING (COUNT(?country) > 2)
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q16068'), ('currencyLabel', 'Deutsche Mark'), ('countries', '9')]
[('currency', 'http://www.wikidata.org/entity/Q201875'), ('currencyLabel', 'kyat'), ('countries', '4')]
[('currency', 'http://www.wikidata.org/entity/Q305981'), ('currencyLabel', 'Abbasi'), ('countries', '5')]
[('currency', 'http://www.wikidata.org/entity/Q173117'), ('currencyLabel', 'Brazilian real'), ('countries', '5')]
[('currency', 'http://www.wikidata.org/entity/Q193094'), ('currencyLabel', 'Iraqi dinar'), ('countries', '8')]
[('currency', 'http://www.wikidata.org/entity/Q476195'), ('currencyLabel', 'Gulf rupee'), ('countries', '3')]
[('currency', 'http://www.wikidata.org/entity/Q572664'), ('currencyLabel', 'dirham'), ('countries', '11')]
[('currency', 'http://www.wikidata.org/entity/Q204992'), ('currencyLabel', 'Italian lira'), ('countries', '8')]
[('currency', 'http://www.wikidata.org/entity/Q2407759'), ('currencyLabel', 'Malaya and British Borneo dollar'), ('c

## Task 7
Consider the currencies obtained in task 6, for each currency return the number of people using that currency (hint: consider the maximum popolation available for each country).

In [39]:
# write your queries
queryString = """ 
SELECT ?p ?name
WHERE {
   wd:Q6256 wdt:P1963 ?p .
   # get the label
   ?p sc:name ?name .
}
LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/P2046'), ('name', 'area')]
[('p', 'http://www.wikidata.org/entity/P3067'), ('name', 'GS1 country code')]
[('p', 'http://www.wikidata.org/entity/P78'), ('name', 'top-level Internet domain')]
[('p', 'http://www.wikidata.org/entity/P935'), ('name', 'Commons gallery')]
[('p', 'http://www.wikidata.org/entity/P30'), ('name', 'continent')]
[('p', 'http://www.wikidata.org/entity/P2936'), ('name', 'language used')]
[('p', 'http://www.wikidata.org/entity/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/entity/P2852'), ('name', 'emergency telephone number')]
[('p', 'http://www.wikidata.org/entity/P47'), ('name', 'shares border with')]
[('p', 'http://www.wikidata.org/entity/P610'), ('name', 'highest point')]
10


P1082 is Population

In [27]:
# write your queries
queryString = """ 
SELECT ?country ?name ?population ?value
WHERE {
   ?country wdt:P31 wd:Q6256 .
   wd:Q6256 wdt:P1082 ?population .
   # get the label
   ?population sc:name ?value .
   ?country sc:name ?name .
}
LIMIT 40
"""
print("Results")
x=run_query(queryString)

Results
Empty


In [28]:
#returning all populations 
queryString = """ 
SELECT ?country ?countryName ?population ?populationValue
WHERE {
    ?country wdt:P31 wd:Q6256 .
    ?country wdt:P1082 ?population .
    # get the label
    ?country sc:name ?countryName .
    ?population sc:name ?populationValue .
}
LIMIT 40
"""
print("Results")
x=run_query(queryString)

Results
Empty


In [38]:
#returning all populations 
queryString = """ 
SELECT ?country ?countryName ?population 
WHERE {
    wd:Q38 wdt:P1082 ?population .
    # get the label
    ?country sc:name ?countryName .
}
LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/P348'), ('countryName', 'software version identifier'), ('population', '60317000')]
[('country', 'http://www.wikidata.org/entity/P4669'), ('countryName', 'tabular software version'), ('population', '60317000')]
[('country', 'http://www.wikidata.org/entity/P747'), ('countryName', 'has edition or translation'), ('population', '60317000')]
[('country', 'http://www.wikidata.org/entity/Q1002728'), ('countryName', 'Fonsi Nieto'), ('population', '60317000')]
[('country', 'http://www.wikidata.org/entity/Q100326534'), ('countryName', "Al Stewart's albums in chronological order"), ('population', '60317000')]
[('country', 'http://www.wikidata.org/entity/Q100327667'), ('countryName', 'Category:Adelina Lopes Vieira'), ('population', '60317000')]
[('country', 'http://www.wikidata.org/entity/Q100343342'), ('countryName', 'Lubomír Stoklásek'), ('population', '60317000')]
[('country', 'http://www.wikidata.org/entity/Q1007576'), ('countryName', 'Ciuda

In [37]:
#returning all populations 
queryString = """ 
SELECT ?country ?countryName ?population 
WHERE {
    ?country wdt:P1082 ?population .
    # get the label
    ?country sc:name ?countryName .
}
LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q10100'), ('countryName', 'Natuna'), ('population', '71448')]
[('country', 'http://www.wikidata.org/entity/Q1018547'), ('countryName', 'Võnnu Rural Municipality'), ('population', '1153')]
[('country', 'http://www.wikidata.org/entity/Q1058869'), ('countryName', 'Nemėžis'), ('population', '2498')]
[('country', 'http://www.wikidata.org/entity/Q109576'), ('countryName', 'Aegadian Islands'), ('population', '4300')]
[('country', 'http://www.wikidata.org/entity/Q113120'), ('countryName', 'Sliven Province'), ('population', '197473')]
[('country', 'http://www.wikidata.org/entity/Q1207964'), ('countryName', 'Kita-ku'), ('population', '163338')]
[('country', 'http://www.wikidata.org/entity/Q13010'), ('countryName', 'Arbúcies'), ('population', '6608')]
[('country', 'http://www.wikidata.org/entity/Q1345562'), ('countryName', 'Toyoyama'), ('population', '15729')]
[('country', 'http://www.wikidata.org/entity/Q1350904'), ('countryName', 'Itano'), ('

In [36]:
queryString = """ 
SELECT ?countryName ?currencyLabel (MAX(?population) AS ?maxP)
WHERE {
    ?country wdt:P31 wd:Q6256 .
    ?country wdt:P38 ?currency .
    ?country wdt:P1082 ?population .
    # get the label
    ?country sc:name ?countryName .
    ?currency sc:name ?currencyLabel .
}
ORDER BY ?currency
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('countryName', 'Kiribati'), ('currencyLabel', 'Kiribati dollar'), ('maxP', '119940')]
[('countryName', 'North Korea'), ('currencyLabel', 'North Korean won'), ('maxP', '25490965')]
[('countryName', 'Canada'), ('currencyLabel', 'Canadian dollar'), ('maxP', '37894799')]
[('countryName', 'Sweden'), ('currencyLabel', 'Swedish krona'), ('maxP', '10409248')]
[('countryName', 'Poland'), ('currencyLabel', 'złoty'), ('maxP', '38382576')]
[('countryName', 'Albania'), ('currencyLabel', 'Albanian lek'), ('maxP', '3020209')]
[('countryName', 'Republic of Artsakh'), ('currencyLabel', 'Armenian dram'), ('maxP', '146600')]
[('countryName', 'Czech Republic'), ('currencyLabel', 'Czech koruna'), ('maxP', '10701777')]
[('countryName', 'Israel'), ('currencyLabel', 'new shekel'), ('maxP', '9093000')]
[('countryName', 'Romania'), ('currencyLabel', 'Romanian leu'), ('maxP', '19586539')]
[('countryName', 'El Salvador'), ('currencyLabel', 'bitcoin'), ('maxP', '6420746')]
[('countryName', 'Norway'), ('c

In [31]:
#returning all populations 
queryString = """ 
SELECT DISTINCT ?countryName ?population
WHERE {
    ?country wdt:P31 wd:Q6256 .
    ?country wdt:P38 wd:Q4916 .
    ?country wdt:P38 ?currency .
    ?country wdt:P1082 ?population .
    # get the label
    ?country sc:name ?countryName .
}
ORDER BY DESC(?population)
LIMIT 40
"""
print("Results")
x=run_query(queryString)

Results
[('countryName', 'France'), ('population', '67063703')]
[('countryName', 'Italy'), ('population', '60317000')]
[('countryName', 'Netherlands'), ('population', '17282163')]
[('countryName', 'Kingdom of the Netherlands'), ('population', '17100715')]
[('countryName', 'Greece'), ('population', '10760421')]
[('countryName', 'Portugal'), ('population', '10347892')]
[('countryName', 'Austria'), ('population', '8809212')]
[('countryName', 'Finland'), ('population', '5501043')]
[('countryName', 'Slovakia'), ('population', '5397036')]
[('countryName', 'Ireland'), ('population', '4761865')]
[('countryName', 'Latvia'), ('population', '1909000')]
[('countryName', 'Cyprus'), ('population', '1141166')]
[('countryName', 'Montenegro'), ('population', '622359')]
[('countryName', 'Malta'), ('population', '465292')]
[('countryName', 'Andorra'), ('population', '78151')]
15


In [2]:
queryString = """ 
SELECT ?currency ?currencyLabel (MAX(?population) AS ?maxP)
WHERE {
    ?country wdt:P31 wd:Q6256 ;
             wdt:P38 ?currency ;
             wdt:P1082 ?population .
    ?currency sc:name ?currencyLabel .
}
GROUP BY ?currency ?currencyLabel   
LIMIT 20
"""
print("Results")
x=run_query(queryString)


Results
[('currency', 'http://www.wikidata.org/entity/Q24302270'), ('currencyLabel', 'ZAAD'), ('maxP', '5780000')]
[('currency', 'http://www.wikidata.org/entity/Q200192'), ('currencyLabel', 'Moroccan dirham'), ('maxP', '36029138')]
[('currency', 'http://www.wikidata.org/entity/Q163712'), ('currencyLabel', 'Malaysian ringgit'), ('maxP', '31624264')]
[('currency', 'http://www.wikidata.org/entity/Q242988'), ('currencyLabel', 'Liberian dollar'), ('maxP', '4731906')]
[('currency', 'http://www.wikidata.org/entity/Q4594'), ('currencyLabel', 'Djiboutian franc'), ('maxP', '956985')]
[('currency', 'http://www.wikidata.org/entity/Q487888'), ('currencyLabel', 'Uzbekistani som'), ('maxP', '33570609')]
[('currency', 'http://www.wikidata.org/entity/Q4598'), ('currencyLabel', 'Ugandan shilling'), ('maxP', '42862958')]
[('currency', 'http://www.wikidata.org/entity/Q208039'), ('currencyLabel', 'Lesotho loti'), ('maxP', '2233339')]
[('currency', 'http://www.wikidata.org/entity/Q731350'), ('currencyLabel'

In [35]:
#returning all populations of countries that use US dollar to check the above query 
queryString = """ 
SELECT DISTINCT ?countryName ?population
WHERE {
    ?country wdt:P31 wd:Q6256 .
    ?country wdt:P38 wd:Q4917 .
    ?country wdt:P38 ?currency .
    ?country wdt:P1082 ?population .
    # get the label
    ?country sc:name ?countryName .
}
ORDER BY DESC(?population)
LIMIT 40
"""
print("Results")
x=run_query(queryString)

Results
[('countryName', 'United States of America'), ('population', '331449281')]
[('countryName', 'Ecuador'), ('population', '16624858')]
[('countryName', 'Zimbabwe'), ('population', '16529904')]
[('countryName', 'El Salvador'), ('population', '6420746')]
[('countryName', 'Liberia'), ('population', '4731906')]
[('countryName', 'East Timor'), ('population', '1296311')]
[('countryName', 'Federated States of Micronesia'), ('population', '105544')]
[('countryName', 'American Samoa'), ('population', '55689')]
[('countryName', 'Northern Mariana Islands'), ('population', '55144')]
[('countryName', 'Marshall Islands'), ('population', '53127')]
[('countryName', 'Palau'), ('population', '21729')]
11


Final query for this task

In [17]:
# write your final query
queryString = """ 
SELECT ?currency ?currencyLabel (MAX(?population) AS ?maxP)
WHERE {
    ?country wdt:P31 wd:Q6256 ;
             wdt:P38 ?currency .
    OPTIONAL {
    ?country wdt:P1082 ?population .
    }
             
    ?currency sc:name ?currencyLabel .
}
GROUP BY ?currency ?currencyLabel   
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q201799'), ('currencyLabel', 'ngultrum'), ('maxP', '807610')]
[('currency', 'http://www.wikidata.org/entity/Q186794'), ('currencyLabel', 'Botswana pula'), ('maxP', '2291661')]
[('currency', 'http://www.wikidata.org/entity/Q211694'), ('currencyLabel', 'Malawian kwacha'), ('maxP', '18622104')]
[('currency', 'http://www.wikidata.org/entity/Q572213'), ('currencyLabel', 'East Timor centavo coins'), ('maxP', '1296311')]
[('currency', 'http://www.wikidata.org/entity/Q199886'), ('currencyLabel', 'Tajikistani somoni'), ('maxP', '8921343')]
[('currency', 'http://www.wikidata.org/entity/Q207514'), ('currencyLabel', 'Paraguayan guaraní'), ('maxP', '6811297')]
[('currency', 'http://www.wikidata.org/entity/Q200753'), ('currencyLabel', 'Mozambican metical'), ('maxP', '29668834')]
[('currency', 'http://www.wikidata.org/entity/Q21531507'), ('currencyLabel', 'Belarusian ruble'), ('maxP', '9349645')]
[('currency', 'http://www.wikidata.org/entity/Q4588